Jupyterbook to create a list of species missing an article in a given language Wikipedia


In [25]:
import math
import json
import requests
import os
import pandas as pd
import mdutils

In [28]:
project = "open-bombus"
license = "cc0,cc-by"
language = "en"

In [29]:
url = f"https://api.inaturalist.org/v1/observations?photo_license={license}&quality_grade=research&per_page=100&project_id={project}"
print(url)

https://api.inaturalist.org/v1/observations?photo_license=cc0,cc-by&quality_grade=research&per_page=100&project_id=open-bombus


In [30]:
result = json.loads(requests.get(url).text)
for page in range(1, math.ceil(result["total_results"]/100)+1):
    try:
        print(page)
        nextpageresult = json.loads(requests.get(url+"&page="+str(page)).text)
    except Exception as e:
        print(e)
        print(url+"&page="+str(page))
        continue
    if "results" in nextpageresult:
        for obs in nextpageresult["results"]:
            result["results"].append(obs)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
Expecting value: line 1 column 1 (char 0)
https://api.inaturalist.org/v1/observations?photo_license=cc0,cc-by&quality_grade=research&per_page=100&project_id=open-bombus&page=162
163
164
Expecting value: line 1 column 1 (char 0)
https://api.inaturalist.org/v1/observations?photo_license=cc0,cc-by&quality_grade=research&per_page=100&project_id=open-bombus&page=164
165
166
Expecting value: line 1 column 1 (char 0)
https://api.inaturalist.org/v1/observations?ph

KeyboardInterrupt: 

In [37]:
table = dict()
for record in result["results"]:
    if record["taxon"]["id"] not in table.keys():
        table[record["taxon"]["id"]] = dict()
    table[record["taxon"]["id"]]["taxon_name"] = record["taxon"]["name"]

to_verify = []
for taxon in table.keys():
    to_verify.append(table[taxon]['taxon_name'])
to_verify

['Bombus ternarius',
 'Bombus lapidarius',
 'Bombus impatiens',
 'Bombus pascuorum',
 'Bombus bimaculatus',
 'Bombus sonorus',
 'Bombus pensylvanicus',
 'Bombus rufocinctus',
 'Bombus pascuorum floralis',
 'Bombus pratorum',
 'Bombus butteli',
 'Bombus terrestris',
 'Bombus griseocollis',
 'Bombus hypnorum',
 'Bombus nevadensis',
 'Bombus transversalis',
 'Bombus insularis',
 'Bombus vosnesenskii',
 'Bombus sandersoni',
 'Bombus citrinus',
 'Bombus humilis',
 'Bombus fervidus',
 'Bombus fraternus',
 'Bombus pauloensis',
 'Bombus melaleucus',
 'Bombus rupestris',
 'Bombus lucorum',
 'Bombus handlirschi',
 'Bombus vagans',
 'Bombus melanopygus',
 'Bombus vancouverensis',
 'Bombus terricola',
 'Bombus appositus',
 'Bombus borealis',
 'Bombus pascuorum vulgo',
 'Bombus terrestris audax',
 'Bombus occidentalis',
 'Pyrobombus',
 'Bombus',
 'Bombus bifarius',
 'Bombus cryptarum',
 'Bombus huntii',
 'Bombus sylvarum',
 'Bombus pascuorum freygessneri',
 'Bombus funebris',
 'Bombus perplexus',
 

In [38]:
from urllib.parse import quote
def verify_wikidata(taxon_names):
    verified = []
    i = 1
    for chunks in [taxon_names[i:i + 50] for i in range(0, len(taxon_names), 50)]:
        names=" ".join('"{0}"'.format(w.replace("\"", "")) for w in chunks)
        query = f"""
             SELECT DISTINCT ?taxon_name (COUNT(?item) AS ?item_count) (COUNT(DISTINCT ?article) AS ?article_count) (GROUP_CONCAT(DISTINCT ?article) as ?wparticles) (GROUP_CONCAT(DISTINCT ?schema) as ?patterns ) WHERE {{
                VALUES ?taxon_name {{{names}}}

                {{
                ## A wikipedia article that is linked through a different item that is labeled as
                ## instance of (P31) organisms known by a particular common name (Q55983715). The link
                ## with the given taxon name is through qualifier of (P642).
                BIND ("organisms known by a particular common name (e:E382)" as ?schema)
                ?qitem wdt:P225 ?taxon_name .
                ?item p:P31 [
                     ps:P31 wd:Q55983715 ;
                     pq:P642 ?qitem ;
                  ] .
                OPTIONAL {{?article schema:about ?item ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
                }}

                UNION

                {{
                ## A wikipedia article that is directly linked through its taxon name
                BIND ("direct link (e:E383)" as ?schema)
                ?item wdt:P225 ?taxon_name .
                OPTIONAL {{?article schema:about ?item ;
                            schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
                }}

                UNION

                {{
                    ## A wikipedia article that is linked through a basionym
                    BIND ("basionym (e:E384)" as ?schema)
                    ?item wdt:P566 ?basionym ;
                          wdt:P225 ?taxon_name .
                    OPTIONAL {{?article schema:about ?basionym ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
                }}

                UNION

                {{
                    ## A wikipedia article that is linked through a taxonomic synonym
                    BIND ("taxon synonym 1 (e:E385)" as ?schema)
                    ?item wdt:P1420 ?synonym ;
                          wdt:P225 ?taxon_name .
                    OPTIONAL {{?article schema:about ?synonym ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
                }}

                UNION

                {{
                  ## A Wikipedia article that is linked through a synonym, this is different from
                  ## the above one where the taxon synonym is through a property taxon synonym
                  ## here it is through a P31 (instance of) synonym.
                    BIND ("taxon synonym 2 (e:E386)" as ?schema)
                   ?qitem wdt:P225 ?taxon_name .
                   ?item p:P31 [
                      ps:P31 wd:Q1040689 ;
                      pq:P642 ?qitem
                   ]
                   OPTIONAL {{?article schema:about ?item ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
                }}

                UNION

                {{
                    ## a wikipedia article that is linked through its protonym
                    BIND ("protonym (e:E387)" as ?schema)
                  ?qitem wdt:P225 ?taxon_name .
                  ?item p:P2868 [
                               ps:P2868 wd:Q10640897 ; # has role protonym
                               pq:P642 ?qitem ;
                         ] .

                OPTIONAL {{?article schema:about ?item ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
                }}

                UNION

                {{
                  ## a wikipedia article that is linked through a property said to be the same as (P460)
                  BIND ("said to be the same (e:E388)" as ?schema)
                  ?item wdt:P225 ?taxon_name .
                  ?wikidata_item wdt:P460 ?item .
                  OPTIONAL {{?article schema:about ?wikidata_item ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}}}
            }} GROUP BY ?taxon_name
            """
        # print(query)
        try:
            results = wdi_core.WDFunctionsEngine.execute_sparql_query(query)

            for result in results["results"]["bindings"]:
                #print(result["taxon_name"], result["article_count"]["value"])
                if int(result["article_count"]["value"])>1:
                    print(result["taxon_name"]["value"], ":", result["article_count"]["value"], result["wparticles"]["value"], result["patterns"]["value"])
                # print(result["article_count"]["value"])
                if result["article_count"]["value"]=='0':
                    verified.append(result["taxon_name"]["value"])
        except Exception as e:
            print(url)
            print(e)
            continue
    return verified

In [40]:
from wikidataintegrator import wdi_core
verified = verify_wikidata(to_verify)
verified

['Bombus terrestris audax',
 'Bombus melaleucus',
 'Bombus handlirschi',
 'Bombus rubicundus',
 'Bombus patagiatus',
 'Bombus zonatus',
 'Bombus diversus',
 'Bombus hortulanus',
 'Bombus soroeensis proteus',
 'Bombus deuteronymus',
 'Bombus fragrans',
 'Bombus schrencki',
 'Bombus volucelloides',
 'Bombus sonani',
 'Bombus bellicosus',
 'Bombus pascuorum mniorum',
 'Bombus opifex',
 'Bombus festivus',
 'Bombus melanurus',
 'Bombus pascuorum sparreanus',
 'Bombus niveatus',
 'Bombus formosellus',
 'Bombus rufofasciatus',
 'Bombus bicoloratus',
 'Bombus haematurus',
 'Bombus saltuarius',
 'Bombus sibiricus',
 'Thoracobombus',
 'Bombus pseudobaicalensis',
 'Bombus tunicatus',
 'Bombus sporadicus',
 'Bombus robustus',
 'Bombus modestus',
 'Bombus laesus',
 'Bombus ladakhensis',
 'Bombus pascuorum moorselensis',
 'Bombus wilmattae',
 'Bombus senex',
 'Bombus excellens']

In [32]:
table = dict()
for record in result["results"]:
    if record['taxon']['rank'] == "species":
        if record["taxon"]["id"] not in table.keys():
            table[record["taxon"]["id"]] = dict()
        table[record["taxon"]["id"]]["taxon_name"] = record["taxon"]["name"]
        for photo in record["observation_photos"]:
            if "photos" not in table[record["taxon"]["id"]].keys():
                table[record["taxon"]["id"]]["photos"] = []
            table[record["taxon"]["id"]]["photos"].append(photo["photo"]["url"])

to_verify = []
for taxon in table.keys():
    to_verify.append(table[taxon]['taxon_name'])


In [41]:
len(verified)

39

In [33]:
to_verify

['Bombus ternarius',
 'Bombus lapidarius',
 'Bombus impatiens',
 'Bombus pascuorum',
 'Bombus bimaculatus',
 'Bombus sonorus',
 'Bombus pensylvanicus',
 'Bombus rufocinctus',
 'Bombus pratorum',
 'Bombus butteli',
 'Bombus terrestris',
 'Bombus griseocollis',
 'Bombus hypnorum',
 'Bombus nevadensis',
 'Bombus transversalis',
 'Bombus insularis',
 'Bombus vosnesenskii',
 'Bombus sandersoni',
 'Bombus citrinus',
 'Bombus humilis',
 'Bombus fervidus',
 'Bombus fraternus',
 'Bombus pauloensis',
 'Bombus melaleucus',
 'Bombus rupestris',
 'Bombus lucorum',
 'Bombus handlirschi',
 'Bombus vagans',
 'Bombus melanopygus',
 'Bombus vancouverensis',
 'Bombus terricola',
 'Bombus appositus',
 'Bombus borealis',
 'Bombus occidentalis',
 'Bombus bifarius',
 'Bombus cryptarum',
 'Bombus huntii',
 'Bombus sylvarum',
 'Bombus funebris',
 'Bombus perplexus',
 'Bombus muscorum',
 'Bombus fragrans',
 'Bombus frigidus',
 'Bombus monticola',
 'Bombus flavifrons',
 'Bombus mckayi',
 'Bombus wurflenii',
 'Bo

In [42]:
import time
photo_collection = []
i = 0
for taxon_name in verified:
    time.sleep(1)
    try:
        photos_url = f"https://api.inaturalist.org/v1/observations?photo_license={license}&project_id={project}&taxon_name={taxon_name}&order=desc&order_by=created_at"
        results = json.loads(requests.get(photos_url).text)
        i+=1
        print(str(i),taxon_name)
        for record in results["results"]:
            if record["taxon"]["id"] not in table.keys():
                continue
            photo_collection.append(record)
            #for photo in record["observation_photos"]:
            #    if "photos" not in table[record["taxon"]["id"]].keys():
            #        table[record["taxon"]["id"]]["photos"] = {}

            #    table[record["taxon"]["id"]]["photos"].append(photo["photo"]["url"])
    except Exception as e:
        print(e)
        print(url+"&page="+str(page))

1 Bombus terrestris audax
2 Bombus melaleucus
3 Bombus handlirschi
4 Bombus rubicundus
5 Bombus patagiatus
6 Bombus zonatus
7 Bombus diversus
8 Bombus hortulanus
9 Bombus soroeensis proteus
10 Bombus deuteronymus
11 Bombus fragrans
12 Bombus schrencki
13 Bombus volucelloides
14 Bombus sonani
15 Bombus bellicosus
16 Bombus pascuorum mniorum
17 Bombus opifex
18 Bombus festivus
19 Bombus melanurus
20 Bombus pascuorum sparreanus
21 Bombus niveatus
22 Bombus formosellus
23 Bombus rufofasciatus
24 Bombus bicoloratus
25 Bombus haematurus
26 Bombus saltuarius
27 Bombus sibiricus
28 Thoracobombus
29 Bombus pseudobaicalensis
30 Bombus tunicatus
31 Bombus sporadicus
32 Bombus robustus
33 Bombus modestus
34 Bombus laesus
35 Bombus ladakhensis
36 Bombus pascuorum moorselensis
37 Bombus wilmattae
38 Bombus senex
39 Bombus excellens


In [43]:
collection = {}
for observation in photo_collection:
    if observation["taxon"]["name"] in verified:
        if observation["taxon"]["id"] not in collection.keys():
            collection[observation["taxon"]["id"]] = dict()
            collection[observation["taxon"]["id"]]["name"] = observation["taxon"]["name"]
            collection[observation["taxon"]["id"]]["observers"] = dict()
        if observation["user"]["id"] not in  collection[observation["taxon"]["id"]]["observers"].keys():
                collection[observation["taxon"]["id"]]["observers"][observation["user"]["id"]] = dict()
        if observation["user"]['name'] != '':
                collection[observation["taxon"]["id"]]["observers"][observation["user"]["id"]]["name"] =  observation["user"]['name']
        if observation["user"]['login'] != '':
                collection[observation["taxon"]["id"]]["observers"][observation["user"]["id"]]["login"] =  observation["user"]['login']

        collection[observation["taxon"]["id"]]["observers"][observation["user"]["id"]]["observations"] = observation["observation_photos"]
collection

{424460: {'name': 'Bombus terrestris audax',
  'observers': {6990462: {'name': 'Anne Taylor',
    'login': 'at54',
    'observations': [{'id': 302284169,
      'position': 0,
      'uuid': 'fb4707b7-5a15-4a91-8c09-02faacf4cdab',
      'photo_id': 324210467,
      'photo': {'id': 324210467,
       'license_code': 'cc0',
       'original_dimensions': {'width': 1850, 'height': 2048},
       'url': 'https://inaturalist-open-data.s3.amazonaws.com/photos/324210467/square.jpg',
       'attribution': 'no rights reserved',
       'flags': [],
       'moderator_actions': [],
       'hidden': False}},
     {'id': 302284165,
      'position': 1,
      'uuid': '93e070b3-e712-43a6-a06d-2083126a0504',
      'photo_id': 324210414,
      'photo': {'id': 324210414,
       'license_code': 'cc0',
       'original_dimensions': {'width': 1536, 'height': 2048},
       'url': 'https://inaturalist-open-data.s3.amazonaws.com/photos/324210414/square.jpg',
       'attribution': 'no rights reserved',
       'flags

In [46]:
def extract_wikidata_taxonomic_schema(name, language="en"):
    query = f"""
     SELECT DISTINCT ?schema ?subject ?subjectLabel ?object ?objectLabel (GROUP_CONCAT(DISTINCT ?object_wikifamily) as ?objectwikis) (GROUP_CONCAT(DISTINCT ?subject_wikifamily) as ?subjectwikis) WHERE {{
       SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
       VALUES ?taxon_name {{"{name}"}}
       # VALUES ?object_wikifamily {{<https://en.wikipedia.org/>}}
       # VALUES ?subject_wikifamily {{<https://en.wikipedia.org/>}}
        {{
        ## A wikipedia article that is linked through a different item that is labeled as
        ## instance of (P31) organisms known by a particular common name (Q55983715). The link
        ## with the given taxon name is through qualifier of (P642).
        BIND ("organisms known by a particular common name (e:E382)" as ?schema)
        BIND (?aka_item as ?subject)
        BIND (?item as ?object)

        optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}

        ?item wdt:P225 ?taxon_name .
        ?aka_item p:P31 [
             ps:P31 wd:Q55983715 ;
             pq:P642 ?item ;
          ] .
        optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}
        }}

        UNION

        {{
            ## A wikipedia article that is linked through a basionym
            BIND ("basionym (e:E384)" as ?schema)
            BIND (?item as ?subject)
            BIND (?basionym as ?object)
            ?item wdt:P566 ?basionym ;
                  wdt:P225 ?taxon_name .
                optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}
        }}

        UNION

        {{
            ## A wikipedia article that is linked through a taxonomic synonym
            BIND ("taxon synonym 1 (e:E385)" as ?schema)
            BIND (?item as ?subject)
            BIND (?synonym as ?object)
            ?item wdt:P1420 ?synonym ;
                  wdt:P225 ?taxon_name .
                optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}
        }}

        UNION

        {{
          ## A Wikipedia article that is linked through a synonym, this is different from
          ## the above one where the taxon synonym is through a property taxon synonym
          ## here it is through a P31 (instance of) synonym.
            BIND ("taxon synonym 2 (e:E386)" as ?schema)
            BIND (?synonym_item as ?subject)
            BIND (?item as ?object)
           ?item wdt:P225 ?taxon_name .
           ?synonym_item p:P31 [
              ps:P31 wd:Q1040689 ;
              pq:P642 ?item
           ]
               optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}
        }}

        UNION

        {{
            ## a wikipedia article that is linked through its protonym
            BIND ("protonym (e:E387)" as ?schema)
            BIND (?protonym_item as ?subject)
            BIND (?item as ?object)
          ?item wdt:P225 ?taxon_name .
          ?protonym_item p:P2868 [
                       ps:P2868 wd:Q10640897 ; # has role protonym
                       pq:P642 ?item ;
                 ] .
                optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}
        }}

        UNION

        {{
          ## a wikipedia article that is linked through a property said to be the same as (P460)
          BIND ("said to be the same (e:E388)" as ?schema)
          BIND (?wikidata_item as ?subject)
          BIND (?item as ?object)
          ?item wdt:P225 ?taxon_name .
          ?wikidata_item wdt:P460 ?item .
              optional {{?subject_article schema:about ?subject ;
                                    schema:isPartOf ?subject_wikifamily .}}
        optional {{?object_article schema:about ?object ;
                                    schema:isPartOf ?object_wikifamily .}}
    }}
    }} GROUP BY ?schema ?subject ?subjectLabel ?object ?objectLabel
    """
    edgesdrawn = []
    df = wdi_core.WDFunctionsEngine.execute_sparql_query(query, as_dataframe=True)
    if len(df) >0:
        schema = graphviz.Digraph(name, filename=f"./wikidata_schemas/{name.replace(' ', '_')}.gv")
        graphviz.Digraph()
        schema.graph_attr['label'] = name
        schema.graph_attr['rankdir'] = 'LR'
        schema.attr('node', shape="rectangle")
        nodes = []
        for index, row in df.iterrows():
            subjectnode = str(row["subject"]).replace("http://www.wikidata.org/entity/", "")
            if subjectnode not in nodes:
                nodes.append(subjectnode)
                schema.node(subjectnode, label=row["subjectLabel"]+' (wd:'+subjectnode+")")

            objectnode = str(row["object"]).replace("http://www.wikidata.org/entity/", "")
            if objectnode not in nodes:
                nodes.append(objectnode)
                schema.node(objectnode, label=row["objectLabel"]+' (wd:'+row["object"].replace("http://www.wikidata.org/entity/", "")+")")

            if f"https://{language}.wikipedia.org/" in row["subjectwikis"].split():
                if subjectnode+nodename not in edgesdrawn:
                    nodename = f"{language}.wiki"
                    schema.node(nodename)
                    schema.edge(subjectnode, nodename, label="schema:isPartOf")
                    edgesdrawn.append(subjectnode+nodename)
            if f"https://{language}.wikipedia.org/" in row["objectwikis"].split():
                 if objectnode+nodename not in edgesdrawn:
                    schema.node(nodename)
                    schema.edge(objectnode, nodename, label="schema:isPartOf")
                    edgesdrawn.append(objectnode+nodename)
            schema.edge(subjectnode, objectnode, label=row["schema"])
        schema.render(f"./wikidata_schemas/{name.replace(' ', '_')}.gv", format="png")
for taxon in collection.keys():
    extract_wikidata_taxonomic_schema(collection[taxon]["name"])

In [50]:
import shutil
shutil.rmtree('./new_articles/')
#shutil.rmtree('./wikidata_schemas/')
os.makedirs(os.path.dirname('./new_articles/'), exist_ok=True)
#os.makedirs(os.path.dirname('./wikidata_schemas/'), exist_ok=True)

In [55]:
for taxon in collection.keys():
    mdFile = mdutils.MdUtils(file_name='new_articles/'+collection[taxon]["name"].replace(" ", "_"),title=collection[taxon]["name"])
    mdFile.new_line(f"[iNaturalist taxon id: {taxon}](https://www.inaturalist.org/taxa/{taxon})")
    schema_file_name = collection[taxon]["name"].replace(" ", "_")+".gv.png"

    if schema_file_name in os.listdir('./wikidata_schemas/'):
        print(schema_file_name,"im here")
        mdFile.new_header(level=1, title="Taxonomy in Wikidata")
        mdFile.new_line(mdFile.new_inline_image(text=collection[taxon]["name"], path="../wikidata_schemas/"+schema_file_name))
    mdFile.new_header(level=1, title="Photos")

    for observer in collection[taxon]["observers"].keys():
        if 'name' in collection[taxon]["observers"][observer].keys():
            print(observer)
            mdFile.new_header(level=2, title="by: "+str(collection[taxon]["observers"][observer]["name"]))

        else:
            mdFile.new_header(level=2, title="by: "+collection[taxon]["observers"][observer]["login"])
        for photo in collection[taxon]["observers"][observer]["observations"]:
            mdFile.new_line(f"![taxon{photo['id']}]({photo['photo']['url'].replace('square', 'medium')})")
    print(mdFile.create_md_file())

6990462
5572020
2883211
357346
2070719
6494569
1708046
1100791
7086217
613693
3565390
4930252
1610962
2371585
5283934
2044851
4499702
2533480
4128246
412180
1486252
487918
1197717
2533480
4128246
491170
1032462
2688869
847297
286069
1699
990532
3976016
2533480
205067
1032462
1378
3338788
1100880
2056691
5476190
3610545
4317444
2752259
3531113
3997065
5427517
2567575
3309653
3182997
4139900
1551755
188597
4985013
1078088
6514553
3606989
1424706
1378
2493157
412180
110005
286069
104632
3362470
1189878
1082621
1229930
475114
990532
1384317
689217
1486252
4522909
2032730
793347
18823
1952440
2501751
1459561
2139481
3997065
2567886
1309693
6331726
1078088
2139481
1980877
3821631
3238846
3239633
3182997
2501751
2920732
2497873
3149353
3309653
3071151
1516707
2511740
3804546
2464103
2499978
2920959
1698191
472396
2055527
1347371
1401655
712306
432605
668245
1570706
2512321
3225167
1255162
788902
1514402
382941
2044920
2843272
4085048
984934
2702752
2861719
2420110
1157928
762185
1570706
61294

In [53]:
collection

{424460: {'name': 'Bombus terrestris audax',
  'observers': {6990462: {'name': 'Anne Taylor',
    'login': 'at54',
    'observations': [{'id': 302284169,
      'position': 0,
      'uuid': 'fb4707b7-5a15-4a91-8c09-02faacf4cdab',
      'photo_id': 324210467,
      'photo': {'id': 324210467,
       'license_code': 'cc0',
       'original_dimensions': {'width': 1850, 'height': 2048},
       'url': 'https://inaturalist-open-data.s3.amazonaws.com/photos/324210467/square.jpg',
       'attribution': 'no rights reserved',
       'flags': [],
       'moderator_actions': [],
       'hidden': False}},
     {'id': 302284165,
      'position': 1,
      'uuid': '93e070b3-e712-43a6-a06d-2083126a0504',
      'photo_id': 324210414,
      'photo': {'id': 324210414,
       'license_code': 'cc0',
       'original_dimensions': {'width': 1536, 'height': 2048},
       'url': 'https://inaturalist-open-data.s3.amazonaws.com/photos/324210414/square.jpg',
       'attribution': 'no rights reserved',
       'flags

In [34]:
def verify_wikidata(taxon_names):
    verified = []
    i = 1
    for chunks in [taxon_names[i:i + 50] for i in range(0, len(taxon_names), 50)]:
        names=" ".join('"{0}"'.format(w) for w in chunks)
        print(chunks)
        query = f"""
             SELECT DISTINCT ?taxon_name (COUNT(?item) AS ?item_count) (COUNT(?article) AS ?article_count)   WHERE {{
                        VALUES ?taxon_name {{{names}}} 
                {{?item p:P31 [
                                ps:P31 wd:Q55983715 ;
                                pq:P642 ?taxon_name ;
                            ] .
                  OPTIONAL {{?article schema:about ?item ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}}}
               UNION
               {{?item wdt:P225 ?taxon_name .
                OPTIONAL {{?article schema:about ?item ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}}}
                 UNION 
               {{?basionym wdt:P566 ?item ;
                          wdt:P225 ?taxon_name .
               OPTIONAL {{?article schema:about ?item ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}}}
               UNION
               {{
                  ?item wdt:P225 ?taxon_name .
                  ?wikidata_item wdt:P460 ?item ;
                  OPTIONAL {{?article schema:about ?wikidata_item ;
                                 schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}
               }}
               UNION
               {{?basionym wdt:P566 ?item .
                ?item wdt:P225 ?taxon_name .
                OPTIONAL {{?article schema:about ?item ;
                         schema:isPartOf 	<{"https://"+language+".wikipedia.org/"}> }}}}
      }} GROUP BY ?taxon_name  
            """

        url = "https://query.wikidata.org/sparql?format=json&query="+query  
        try:
            results = json.loads(requests.get(url).text)
        except:
            print(error)
            continue
        for result in results["results"]["bindings"]:
            if result["article_count"]["value"]=='0':
                verified.append(result["taxon_name"]["value"])
    return verified

In [35]:
erified = verify_wikidata(to_verify)
verified

['Bombus ternarius', 'Bombus lapidarius', 'Bombus impatiens', 'Bombus pascuorum', 'Bombus bimaculatus', 'Bombus sonorus', 'Bombus pensylvanicus', 'Bombus rufocinctus', 'Bombus pratorum', 'Bombus butteli', 'Bombus terrestris', 'Bombus griseocollis', 'Bombus hypnorum', 'Bombus nevadensis', 'Bombus transversalis', 'Bombus insularis', 'Bombus vosnesenskii', 'Bombus sandersoni', 'Bombus citrinus', 'Bombus humilis', 'Bombus fervidus', 'Bombus fraternus', 'Bombus pauloensis', 'Bombus melaleucus', 'Bombus rupestris', 'Bombus lucorum', 'Bombus handlirschi', 'Bombus vagans', 'Bombus melanopygus', 'Bombus vancouverensis', 'Bombus terricola', 'Bombus appositus', 'Bombus borealis', 'Bombus occidentalis', 'Bombus bifarius', 'Bombus cryptarum', 'Bombus huntii', 'Bombus sylvarum', 'Bombus funebris', 'Bombus perplexus', 'Bombus muscorum', 'Bombus fragrans', 'Bombus frigidus', 'Bombus monticola', 'Bombus flavifrons', 'Bombus mckayi', 'Bombus wurflenii', 'Bombus sylvicola', 'Bombus kluanensis', 'Bombus c

[]

In [36]:
table

In [ ]:
missing_en_wp_by_user = pd.DataFrame(columns=["taxon", "iNaturalist_taxon_id", "iNaturalist_observation_id", "reviewers"])

for observation in result["results"]:
    if observation["taxon"]["name"] in verified:
        missing_en_wp_by_user.loc[len(missing_en_wp_by_user)] = [observation["taxon"]["name"], observation["taxon"]["id"], observation["id"], len(observation["reviewed_by"])]
missing_en_wp_by_user

In [ ]:
import shutil
shutil.rmtree('./new_articles/')
os.makedirs(os.path.dirname('./new_articles/'), exist_ok=True)

In [ ]:
for taxon in table.keys():
    if table[taxon]["taxon_name"] in verified:
        mdFile = mdutils.MdUtils(file_name='new_articles/'+table[taxon]["taxon_name"].replace(" ", "_"),title=table[taxon]["taxon_name"])
        mdFile.new_line(f"[iNaturalist taxon id: {taxon}](https://www.inaturalist.org/taxa/{taxon})")
        for image in table[taxon]["photos"]:
            mdFile.new_line(f"![{table[taxon]['taxon_name']}]({image.replace('square', 'medium')})")
        mdFile.create_md_file()

In [ ]:
import yaml

with open(r'_toc.yml') as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    species = yaml.load(file, Loader=yaml.FullLoader)

species["parts"][0]["chapters"] = []
for file in os.listdir('./new_articles/'):
    species["parts"][0]["chapters"].append({'file': 'new_articles/'+file})
with open(r'_toc.yml', 'w+') as file:
    documents = yaml.dump(species, file)

In [ ]:
from ghp_import import ghp_import
ghp_import("_build/html", nojekyll=True, push=True, force=True)